In [1]:
import pickle
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.
/Users/joannasyeow/anaconda/envs/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=0.,
      width_shift_range=0.,
      height_shift_range=0.,
      shear_range=0.,
      zoom_range=0.,
      horizontal_flip=False,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('./train/image/', target_size = (128,32), batch_size = 20, 
                                                    class_mode = 'categorical')

Found 45891 images belonging to 30 classes.


In [3]:
learning_rate = 0.01
training_iters = 5000
batch_size = 20
display_step = 2
n_input = 4096 # we unravel the 28x28 image into a row of 784 entries
n_classes = 8 # the end state is the likelihood of falling into 1 of 8 classes
dropout = 0.75 # likelihood that a given entity will be kept

In [4]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [5]:
def conv2d(x, W, b, strides=1):
    # Takes in an x, a set of weights, strides over the image, and with 0 pads around the border
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME') 
    # Add in a bias term
    x = tf.nn.bias_add(x, b)
    # ReLU activation
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # Takes the max value over the window to reduce size
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], 
            strides=[1, k, k, 1],
            padding='SAME')

In [6]:
def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape=[-1, 128, 32, 3])

    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    
    fc1 = tf.reshape(conv1, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])

    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [14]:
weights = {
        'wc1': tf.Variable(tf.random_normal([8, 8, 3, 32])),
        'wd1': tf.Variable(tf.random_normal([16*4*128, 1024])),
        'out': tf.Variable(tf.random_normal([1024, n_classes]))
        }

biases = {
        'bc1': tf.Variable(tf.random_normal([32])),
        'bd1': tf.Variable(tf.random_normal([1024])),
        'out': tf.Variable(tf.random_normal([n_classes]))
        }

In [15]:
pred = conv_net(x, weights, biases, keep_prob)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [16]:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [17]:
accuracy_list = []

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:
        batch_x, batch_y = train_generator.next()
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

        if step % display_step == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                y: batch_y, keep_prob: 1.})

            print ("Iteration", str(step*batch_size), "and Minibatch Loss= ", \
                    "{:.6f}".format(loss), "and Training Acc.= ", \
                    "{:.5f}".format(acc))
            
            accuracy_list.append((step*batch_size, loss, acc))
        step += 1

#     print "Test accuracy:", \
#             sess.run(accuracy, feed_dict={x: mnist.test.images,
#                 y: mnist.test.labels,
#                 keep_prob: 1.})

#     y_actual = tf.argmax(y, 1).eval(feed_dict={y:mnist.test.labels})
#     y_pred = tf.argmax(pred, 1).eval(feed_dict={x:mnist.test.images, keep_prob:1.})

#     print pd.DataFrame(confusion_matrix(y_actual, y_pred),
#             index=['Actually %s' % x for x in range(0, 10)],
#             columns=['Predicted %s' %  x for x in range(0, 10)])

#     print classification_report(y_actual, y_pred)
    
#     plt.plot([x[0] for x in accuracy_list],
#          [x[1] for x in accuracy_list],
#         )
#     plt.title('Loss at iterations')
#     plt.show()

#     plt.plot([x[0] for x in accuracy_list],
#              [x[2] for x in accuracy_list],
#             )
#     plt.title('Accuracy at iterations')
#     plt.show()


ValueError: Cannot feed value of shape (20, 128, 32, 3) for Tensor 'Placeholder:0', which has shape '(?, 4096)'